In [ ]:
import keras_cv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# TEST

In [ ]:
#from diffusion_models.stable_diffusion import StableDiffusion
from keras_cv.models import StableDiffusion

In [ ]:

model = StableDiffusion(img_width=512, img_height=512, jit_compile=False)


In [ ]:

from utils import crop_and_scale_image, show_image, get_image, normalize_m1to1, normalize_0to1

img_path = r"images\lion_savannah.jpeg"

image = crop_and_scale_image(get_image(img_path))

plt.imshow(image)
image

In [ ]:
#image = normalize_m1to1(image)
image = normalize_0to1(image)
images = tf.expand_dims(image, axis=0) # encoder expects a tensor of shape (batch_size, height, width, channels) so we add a batch dimension


In [ ]:
image_encoder = model.image_encoder
#image_encoder.summary()


In [ ]:
image_encoded_latent = image_encoder(images, training=False)

In [ ]:

# Plotting the latents
fig, axs = plt.subplots(1, 4, figsize=(20, 5))  # 1 row, 4 columns

# Loop over each channel
for i in range(4):
    channel = image_encoded_latent[0, :, :, i]
    axs[i].imshow(channel, cmap="gray")
    axs[i].set_title(f"Channel {i+1}")
    axs[i].axis("off")  # Hide axis

fig.suptitle("Latents")
plt.show()

In [ ]:
image_decoder = model.decoder #64x64x4 --> 512x512x3    
image_decoder.summary()

In [ ]:
image_decoded = image_decoder(image_encoded_latent, training=False)
image_decoded

In [ ]:
plt.imshow(image_decoded[0])    

In [ ]:
text_encoder = model.text_encoder
text_encoder.summary()

In [ ]:
tokenizer = model.tokenizer
tokenizer

In [ ]:
prompt = ""
tokens = tokenizer.encode(prompt)
tokens

In [ ]:
tokens = tokens + [49407] * (77 - len(tokens)) #fill up with padding tokens so that the length is 77

In [ ]:
model._get_pos_ids()

In [ ]:
text_encoder_inputs = tf.expand_dims(tokens, 0) # encoder expects a tensor of shape (batch_size, seq_length) so we add a batch dimension

context = text_encoder.predict_on_batch({"tokens": text_encoder_inputs, "positions": model._get_pos_ids()})
context

In [ ]:
#keras_cv.models.stable_diffusion.DiffusionModel
ldm = model.diffusion_model

In [ ]:
ldm.summary()

In [ ]:
timestep = 1
t_emb = model._get_timestep_embedding(timestep=timestep, batch_size=1)
t_emb

In [ ]:
images_latent = tf.expand_dims(image_encoded_latent, axis=0)
images_latent

In [ ]:
output = ldm.predict_on_batch({'latent':image_encoded_latent, 
                      'timestep_embedding':t_emb, 
                      'context':context})     

In [ ]:
output.shape # (1, 64, 64, 4)